# Sales Analysis

---------------------

## Import Necessary Libraries

In [2]:
import pandas as pd 
import os

#### Merging 12 months of sales data into one sigle file

In [3]:
files = [file for file in os.listdir('./Sales_Data')]

complete_data = pd.DataFrame()

for file in files:
    data = pd.read_csv("./Sales_Data/"+file)
    complete_data = pd.concat([complete_data,data])
complete_data.to_csv('All_months.csv', index = False)

In [4]:
df = pd.read_csv('All_months.csv')
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,278797,Wired Headphones,1,11.99,11/21/19 09:54,"46 Park St, New York City, NY 10001"
1,278798,USB-C Charging Cable,2,11.95,11/17/19 10:03,"962 Hickory St, Austin, TX 73301"
2,278799,Apple Airpods Headphones,1,150.0,11/19/19 14:56,"464 Cherry St, Los Angeles, CA 90001"
3,278800,27in FHD Monitor,1,149.99,11/25/19 22:24,"649 10th St, Seattle, WA 98101"
4,278801,Bose SoundSport Headphones,1,99.99,11/09/19 13:56,"522 Hill St, Boston, MA 02215"


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
Order ID            186305 non-null object
Product             186305 non-null object
Quantity Ordered    186305 non-null object
Price Each          186305 non-null object
Order_Date          186305 non-null object
Purchase Address    186305 non-null object
dtypes: object(6)
memory usage: 8.6+ MB


### Augment Data with additional columns

In [12]:
df = df.rename(columns = {'Order Date' : 'Order_Date'})

In [15]:
df['Month'] = df.Order_Date.apply([lambda x: x.split('/')[0]])

AttributeError: 'Series' object has no attribute 'split'